# Does the Common Swift's tune behave in a Zipf-like pattern? 

## A vague concept and analysis

## The Common Swift

The common swift (_Apus apus_), is a medium-sized bird resembling the barn swallow and house martin but larger, belonging to the order Apodiformes. These resemblances result from convergent evolution, as their closest relatives are New World hummingbirds and Southeast Asian treeswifts. Its scientific name "Apus" is derived from Latin, meaning swift, reflecting the ancient belief that swifts had no feet. Swifts have short legs used for clinging to vertical surfaces, and they avoid settling on the ground to minimize vulnerability, with non-breeding individuals often in continuous flight for up to ten months.

![Swift Pretty](images/swift-flying.webp)

### Taxonomy and Physiology

The common swift, was classified by Carl Linnaeus in 1758 as _Hirundo apus_ and later categorized under the _genus Apus_ by Giovanni Antonio Scopoli in 1777. Its name "apus" is derived from the Latin word for swift, reflecting the belief that these birds were footless swallows. These birds measure 16–17 cm in length with a wingspan of 38–40 cm. They are primarily blackish-brown, with a small white or pale grey chin patch that is not easily visible from a distance.


### Language Behavior
Common swifts are known for their distinctive loud screaming calls, often forming groups during summer evenings. These gatherings may serve various purposes, including ascending to sleep in flight, with radar tracking revealing flocking behavior during specific times, possibly for social interaction or information exchange. 

### Final Motivations
As the name would led you to believe, the Common Swift is a fairly popular recorded species, specifically in the Xenocanto bird dataset there are thousands of high quality recordings that can be obtained through a wrapped API (see _example_download.py_) therefore this is a good starter specie to test if Zipf's law is present after some preprocessing.

The main focus of this work will be to explore a novel way to synthetize bird sounds into a codified language and thus allowing possible languange analysis in the future.

### Xenocanto Dataset 

_To download the dataset using the wrapped API (pypi xenocanto) run example_download.py_


#### Check Metadata

In [39]:
# check metadata files in the dataset
import os 
import json


bird = "CommonSwift"

# check a metadata file's content
path = f"./dataset/metadata/{bird}/"
metalist = os.listdir(path)

# open and read the JSON file
chosen_meta = metalist[0]
with open(f"./dataset/metadata/CommonSwift/{chosen_meta}", 'r') as json_file:
    metadata = json.load(json_file)

#print(metadata)


So each metadata file has a ton of recordings with fields that _could_ be interesting if data's rich enough, let's visualize
how rich is our data by merging both metadata files to a pandas dataframe and getting some statistics.

In [40]:
# uses: metalist from cell 9 and imports
# merge metadata files into one and then dataframe it
import pandas as pd


records = []

for meta in metalist:
    # open each metadata file and load dict
    with open(f"./dataset/metadata/CommonSwift/{meta}", 'r') as json_file:
        metadata = json.load(json_file)
        # we need only the recordings from json
        sub_records = metadata['recordings']
        for record in sub_records:
            records.append(record)

# now we create the dataframe
df = pd.DataFrame(records)
df.sample(5)

,id,gen,sp,ssp,group,en,rec,cnt,loc,lat,...,rmk,bird-seen,animal-seen,playback-used,temp,regnr,auto,dvc,mic,smp
458,799897,Apus,apus,,birds,Common Swift,julien Birard,France,Arles centre - Elie Giraud,43.6776,...,,no,no,no,,,yes,Zoom H4N,Micro Omni Oktava MK-012,44100
678,376914,Apus,apus,,birds,Common Swift,Stanislas Wroza,France,"Hyères, Var, Provence-Alpes-Côte d'Azur",42.992,...,,yes,yes,no,,,no,,,44100
146,371024,Apus,apus,,birds,Common Swift,Stanislas Wroza,France,"Hyères, Var, Provence-Alpes-Côte d'Azur",43.0671,...,,yes,yes,no,,,no,,,44100
567,574526,Apus,apus,,birds,Common Swift,Gary Elton,United Kingdom,"Great Britain (near Great Massingham), Norfol...",52.7759,...,,yes,yes,no,,,no,,,44100
55,655019,Apus,apus,,birds,Common Swift,Ad Hilders,Netherlands,"Roosendaal, Roosendaal, Noord-Brabant",51.5121,...,,no,no,no,,,no,,,44100


Clearly, there are some useless columns so let's ignore those. Some columns that could be interesting may not be rich enough.

TODO: Check later if we can separate by male and female.

In [41]:
# clean a bit
uselful_cols = ['id', 'cnt', 'lat', 'lng', 'type', 'q', 'length', 'time', 'date', 'bird-seen', 'smp']

df[uselful_cols].sample(10)

,id,cnt,lat,lng,type,q,length,time,date,bird-seen,smp
312,664245,Netherlands,51.9892,4.4694,"call, flight call",A,0:18,22:01,2021-07-20,yes,44100
357,546693,Spain,41.1684,0.974,flight call,A,0:08,21:00,2020-04-16,no,44100
149,368207,Ukraine,49.5345,26.2093,flight call,C,0:13,10:32,2017-05-07,no,48000
133,377514,Spain,37.8738,-6.6203,flight call,C,0:21,?,2017-05-01,yes,44100
710,269755,Russian Federation,55.8697,48.0544,flight call,B,3:45,10:30,2015-07-18,yes,44100
495,734771,France,44.0632,1.9509,"flight call, wing flaps",B,1:05,06:30,2022-06-29,no,44100
57,655010,Netherlands,51.5121,4.4359,"call, flight call",C,0:10,20:07,2021-05-29,yes,44100
181,37174,Czech Republic,49.4556,17.4501,flight call,C,0:25,20:45,2009-07-16,unknown,44100
79,571911,Spain,41.0871,1.1009,call,C,0:11,09:00,2020-06-24,yes,44100
261,809365,Netherlands,51.9892,4.4694,"call, flight call",A,0:05,20:18,2023-06-16,no,44100


In [42]:
# check statistics
import plotly.express as px

px.histogram(df[uselful_cols], x='cnt', template='plotly_dark', 
             height=400, width=600,
             title='Recordings by Country')

In [43]:
px.histogram(df[uselful_cols], x='q', template='plotly_dark', category_orders=dict(q=['A', 'B', 'C', 'D']),
             height=400, width=600,
             title='Recordings by Quality', histnorm='percent')

In [46]:
# clean by seconds
def time_to_seconds(time_str):
    minutes, seconds = map(int, time_str.split(':'))
    return minutes * 60 + seconds

df['seconds'] = df['length'].apply(time_to_seconds)
px.histogram(df[df.seconds <= 120], x='seconds', cumulative=True,
             height=400, width=600,
             template='plotly_dark', title='Recordings by Duration [s] (Cumulative)', histnorm='percent')

So if we use recordings that are 20 seconds or less we have 60% of data, and of at least C quality we have 80% of data. If these values are not correlated in some way we should expect to have around 48% of data as a available to process (being picky).

We also take only sample rates of 44khz because it makes our life easier.

In [47]:
# create a sub dataset with the picky data
dataset = df[uselful_cols + ['seconds']][(df.seconds <= 20) & ((df.q != 'D') | (df.q != 'E')) & (df.smp == '44100')].copy().reset_index(drop=True)
dataset

,id,cnt,lat,lng,type,q,length,time,date,bird-seen,smp,seconds
0,184143,Switzerland,46.9509,7.4339,flight call,B,0:02,11:30,2014-06-28,yes,44100,2
1,182340,Germany,None,None,flight call,B,0:09,?,2014-06-16,yes,44100,9
2,179334,Spain,37.3811,-5.947,flight call,B,0:18,?,2014-05-12,yes,44100,18
3,144363,Israel,31.7833,35.2167,call,B,0:07,11:00,2011-04-14,no,44100,7
4,140595,Spain,42.0029,-5.6742,flight call,B,0:12,08:30,2013-06-25,yes,44100,12
...,...,...,...,...,...,...,...,...,...,...,...,...
300,199124,Russian Federation,56.0818,47.2859,flight call,B,0:18,19:00,2014-05-29,yes,44100,18
301,198310,Russian Federation,56.1545,47.2399,call,B,0:11,09:30,2014-06-01,yes,44100,11
302,198309,Russian Federation,56.0814,47.3027,call,B,0:19,09:30,2014-06-01,yes,44100,19
303,198308,Russian Federation,56.0814,47.3027,call,B,0:12,09:30,2014-06-01,yes,44100,12


### Frequency Spectrum of recordings

We will observe some frequency spectra first to have an idea of the data, first we test the spectrogram. 

In [53]:
# test spectrogram function from scipy
from scipy.signal import spectrogram
import numpy as np
import plotly.graph_objects as go


N = 1000
f = 50 # 50 hz -> wave = 2*pi*f*t
t = np.linspace(0, 5, N)
y = np.sin(2 * np.pi * f * t)

_f, _t, _S = spectrogram(y, f)

# Create the heatmap
fig = go.Figure(data=go.Heatmap(
    x=_t,
    y=_f,
    z=_S,
    colorscale='Viridis',  # You can choose any other color scale
))

# Add labels to the axes (optional)
fig.update_xaxes(title='X Axis Title')
fig.update_yaxes(title='Y Axis Title')

# Set the title of the heatmap
fig.update_layout(
    title='Test heatmap with a Sin wave (1 frequency)',
    # dark mode, a mess beacuse it's not plotly express
    paper_bgcolor='black',
    plot_bgcolor='black',
    # Change font color to white
    font=dict(color='white'),
    # Change grid color to gray
    xaxis=dict(showgrid=True, gridcolor='gray'),
    yaxis=dict(showgrid=True, gridcolor='gray'),
    width=600,
    height=400
)

# Show the plot
fig.show()


Now let's test loading a sound file!

In [55]:
# TODO using librosa